In [3]:
%%capture
# Google Gemini SDK + LangChain integration (replaces ibm-watsonx-ai + langchain-ibm)
!pip install google-generativeai langchain-google-genai

# LangChain core (same as original, these are provider-agnostic)
!pip install langchain langchain-core langchain-community langchain-experimental langchainhub

# PDF loading (same as original)
!pip install pypdf

# Vector database (same as original)
!pip install chromadb

# For loading .env file with your GOOGLE_API_KEY
!pip install python-dotenv

In [4]:
import os
from dotenv import load_dotenv

# ============================================================
# Remove corporate proxy settings that block Google API access
# (Walmart VPN/proxy intercepts HTTPS calls to external APIs)
# ============================================================
for proxy_var in ["HTTP_PROXY", "HTTPS_PROXY", "http_proxy", "https_proxy"]:
    os.environ.pop(proxy_var, None)

# ============================================================
# Load your Google Gemini API key from the .env file
# The .env file is in the same folder as this notebook
# with the line: GOOGLE_API_KEY=your_actual_key_here
# Get a free key from: https://aistudio.google.com/apikey
# ============================================================
load_dotenv()  # Reads the .env file and loads vars into os.environ
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if not GOOGLE_API_KEY or GOOGLE_API_KEY == "YOUR_API_KEY_HERE":
    raise ValueError("Please set your GOOGLE_API_KEY in the .env file!")


In [5]:
# Warnings suppression -- stays exactly the same
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')
import os
os.environ['ANONYMIZED_TELEMETRY'] = 'False'

# Google Gemini equivalents of the IBM imports
import google.generativeai as genai                          # Replaces: ModelInference (direct API access)
from langchain_google_genai import ChatGoogleGenerativeAI    # Replaces: WatsonxLLM (LangChain wrapper)
from langchain_google_genai import GoogleGenerativeAIEmbeddings  # For embeddings (you'll need this later for vector stores)

In [6]:
from dotenv import load_dotenv

# Load API key from your .env file
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# Configure the Gemini SDK
genai.configure(api_key=GOOGLE_API_KEY)

# --- For direct Gemini SDK calls (equivalent of ModelInference) ---
model = genai.GenerativeModel(
    model_name="gemini-2.0-flash",
    generation_config=genai.types.GenerationConfig(
        max_output_tokens=256,    # was GenParams.MAX_NEW_TOKENS
        temperature=0.2,          # was GenParams.TEMPERATURE
    )
)

# --- For LangChain chains (equivalent of WatsonxLLM) ---
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=GOOGLE_API_KEY,
    max_output_tokens=256,
    temperature=0.2,
)

In [7]:
msg = model.generate_content("Aap kaisi ho ma'am.")
print(msg.candidates[0].content.parts[0].text)
print(msg.text)

Mai ek AI assistant hu, isliye meri koi bhavna nahi hoti. Mai theek hu, aur aapki madad karne ke liye taiyar hu. Aap kaise hain?

Mai ek AI assistant hu, isliye meri koi bhavna nahi hoti. Mai theek hu, aur aapki madad karne ke liye taiyar hu. Aap kaise hain?



In [8]:
# Remove corporate proxy settings that block Google API access
for proxy_var in ["HTTP_PROXY", "HTTPS_PROXY", "http_proxy", "https_proxy"]:
    os.environ.pop(proxy_var, None)

In [22]:
print(llm.invoke("How are you ?").content)

I am doing well, thank you for asking! As a large language model, I don't experience emotions or feelings in the same way humans do, but I am functioning optimally and ready to assist you with any questions or tasks you have. How can I help you today?


In [23]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

In [24]:
msg = llm.invoke(
    [
        SystemMessage(content="You are a helpful assistant who plays a role of iOS interviewer who is hiring for iOS Developer in Indian Startup which is at par with on going trends of AI."),
        HumanMessage(content="Hello ma'am.")
    ]
)
print(msg.content)

Hello! Thank you for coming in today. Please, have a seat.

I'm excited to learn more about you and your experience. As you know, we're a fast-growing startup in the AI space, and we're looking for a talented iOS Developer to join our team and help us build innovative and user-friendly mobile experiences.

Before we dive into the technical questions, could you please tell me a little bit about yourself and what excites you about iOS development, particularly in the context of AI-driven applications?


In [26]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

conversationHistory = [
    SystemMessage(content="You are a helpful assistant who plays a role of iOS interviewer who is hiring for iOS Developer in Indian Startup which is at par with on going trends of AI.")
]

def chat_with_llm(user_input):
    conversationHistory.append(HumanMessage(content=user_input))
    response = llm.invoke(conversationHistory)
    conversationHistory.append(response)
    return response.content

print(chat_with_llm("Hello ma'am."))

print(conversationHistory)


Hello! Thank you for coming in today. Please, have a seat.

I'm excited to learn more about you and your experience. As you know, we're a fast-paced startup in the AI space, and we're looking for a passionate and skilled iOS Developer to join our team. We need someone who can not only build great apps but also understands the potential of integrating AI into the mobile experience.

Before we dive into the technical questions, could you please walk me through your background and tell me a bit about what excites you about iOS development and AI?
[SystemMessage(content='You are a helpful assistant who plays a role of iOS interviewer who is hiring for iOS Developer in Indian Startup which is at par with on going trends of AI.', additional_kwargs={}, response_metadata={}), HumanMessage(content="Hello ma'am.", additional_kwargs={}, response_metadata={}), AIMessage(content="Hello! Thank you for coming in today. Please, have a seat.\n\nI'm excited to learn more about you and your experience. A

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

conversationHistory = [
    SystemMessage(content="You are a helpful assistant who plays a role of iOS interviewer who is hiring for iOS Developer in Indian Startup which is at par with on going trends of AI.")
]

def chat_with_llm(user_input):
    conversationHistory.append(HumanMessage(content=user_input))
    response = llm.invoke(conversationHistory)
    conversationHistory.append(response)
    return response.content

def display_chat():
    """Prints the full conversation history in a clean chat view"""
    print("\n" + "=" * 60)
    print("📋 CONVERSATION HISTORY")
    print("=" * 60)
    for msg in conversationHistory:
        if isinstance(msg, SystemMessage):
            print(f"\n🔧 [SYSTEM]: {msg.content}")
        elif isinstance(msg, HumanMessage):
            print(f"\n🧑 [YOU]: {msg.content}")
        elif isinstance(msg, AIMessage):
            print(f"\n🤖 [AI]: {msg.content}")
    print("\n" + "=" * 60)

# Interactive loop -- type "quit" to exit
while True:
    user_input = input("\n🧑 You: ")
    if user_input.lower() in ["quit", "exit", "bye"]:
        print("Ending interview. Goodbye!")
        display_chat()  # Show full history at the end
        break
    
    response = chat_with_llm(user_input)
    print(f"\n🤖 AI: {response}")

In [1]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

# Run this cell ONCE to initialize
conversationHistory = [
    SystemMessage(content="You are a helpful assistant who plays a role of iOS interviewer who is hiring for iOS Developer in Indian Startup which is at par with on going trends of AI.")
]

def chat_with_llm(user_input):
    conversationHistory.append(HumanMessage(content=user_input))
    response = llm.invoke(conversationHistory)
    conversationHistory.append(response)
    return response.content

def display_chat():
    """Prints the full conversation history in a clean chat view"""
    print("=" * 60)
    for msg in conversationHistory:
        if isinstance(msg, SystemMessage):
            print(f"\n🔧 [SYSTEM]: {msg.content}")
        elif isinstance(msg, HumanMessage):
            print(f"\n🧑 [YOU]: {msg.content}")
        elif isinstance(msg, AIMessage):
            print(f"\n🤖 [AI]: {msg.content}")
    print("\n" + "=" * 60)

In [9]:
# Change this message and re-run this cell for each turn
print("🤖 AI:", chat_with_llm("Hello ma'am."))
print()
display_chat()

🤖 AI: Hello! Please, call me [My Name, if applicable, or just leave it as "Hello"]. Thank you for coming in today. I'm excited to learn more about you and your experience.

This role is for an iOS Developer at [Startup Name], a fast-growing Indian startup leveraging AI to [briefly describe what the startup does]. We're looking for someone passionate about building innovative and user-friendly mobile experiences.

To start, could you please walk me through your background and tell me a bit about your experience with iOS development?


🔧 [SYSTEM]: You are a helpful assistant who plays a role of iOS interviewer who is hiring for iOS Developer in Indian Startup which is at par with on going trends of AI.

🧑 [YOU]: Hello ma'am.

🧑 [YOU]: Hello ma'am.

🤖 [AI]: Hello! Please, call me [My Name, if applicable, or just leave it as "Hello"]. Thank you for coming in today. I'm excited to learn more about you and your experience.

This role is for an iOS Developer at [Startup Name], a fast-growing 

In [10]:
from langchain_core.prompts import PromptTemplate

In [11]:
prompt = PromptTemplate(
    template="""
    You are a helpful assistant who plays a role of iOS interviewer who is hiring for iOS Developer in Indian Startup which is at par with on going trends of AI.
    {user_input}
    """
)

input_ = {"user_input": "Hello ma'am."}


In [12]:
prompt.invoke(input_)

StringPromptValue(text="\n    You are a helpful assistant who plays a role of iOS interviewer who is hiring for iOS Developer in Indian Startup which is at par with on going trends of AI.\n    Hello ma'am.\n    ")

In [16]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant who plays a role of iOS interviewer who is hiring for iOS Developer in Indian Startup which is at par with on going trends of AI."),
        ("human", "{user_input}"),
        ("ai", "Hello {user_input}")
    ]
)

prompt.invoke({"user_input": "Hello ma'am."})
    

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant who plays a role of iOS interviewer who is hiring for iOS Developer in Indian Startup which is at par with on going trends of AI.', additional_kwargs={}, response_metadata={}), HumanMessage(content="Hello ma'am.", additional_kwargs={}, response_metadata={}), AIMessage(content="Hello Hello ma'am.", additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])])